In [3]:
# =================== IMPORTS ===================
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models, regularizers, Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# =================== MOUNT DRIVE ===================
from google.colab import drive
drive.mount('/content/drive')

# =================== SETUP ===================
dataset_path = "/content/drive/MyDrive/NNDL Dataset/Dataset"
categories = [
    "gond painting", "kalighat painting", "kangra painting", "kerala mural",
    "madhubani painting", "mandana art drawing", "pichwai painting", "warli painting"
]
img_size = (128, 128)
valid_extensions = (".jpg", ".jpeg", ".png")

# =================== LOAD IMAGES ===================
def load_images(folder_path, img_size):
    images, labels = [], []
    for category in categories:
        category_path = os.path.join(folder_path, category)
        if not os.path.exists(category_path):
            print(f"⚠️ Warning: {category_path} does not exist. Skipping.")
            continue
        for filename in os.listdir(category_path):
            if filename.lower().endswith(valid_extensions):
                img_path = os.path.join(category_path, filename)
                img = load_img(img_path, target_size=img_size)
                img_array = img_to_array(img) / 255.0
                images.append(img_array)
                labels.append(categories.index(category))
    return np.array(images), np.array(labels)

X, y = load_images(dataset_path, img_size)
if len(X) == 0:
    raise ValueError("No images were loaded. Please check your dataset folder structure.")
print(f"✅ Loaded {len(X)} images with shape: {X.shape}")

# =================== SPLIT DATA ===================
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# =================== CNN MODEL ===================
def build_cnn_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3),
                      kernel_regularizer=regularizers.l2(0.001)),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(len(categories), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# =================== EVALUATION FUNCTION ===================
def evaluate_model(model, X_val, y_val):
    y_pred = np.argmax(model.predict(X_val), axis=1)
    print("Accuracy :", accuracy_score(y_val, y_pred))
    print("Precision:", precision_score(y_val, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_val, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_val, y_pred, average='weighted'))

# =================== RNN PREPROCESSING ===================
X_rnn = X.reshape(-1, 128, 128 * 3)
X_train_rnn, X_val_rnn = train_test_split(X_rnn, test_size=0.2, stratify=y, random_state=42)

# =================== RNN MODEL ===================
def build_rnn_model():
    inputs = layers.Input(shape=(128, 384))
    x = layers.SimpleRNN(64)(inputs)
    outputs = layers.Dense(len(categories), activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# =================== BIDIRECTIONAL RNN ===================
def build_bidirectional_rnn_model():
    inputs = layers.Input(shape=(128, 384))
    x = layers.Bidirectional(layers.SimpleRNN(64))(inputs)
    outputs = layers.Dense(len(categories), activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# =================== LSTM MODEL ===================
def build_lstm_model():
    inputs = layers.Input(shape=(128, 384))
    x = layers.Bidirectional(layers.LSTM(64))(inputs)
    outputs = layers.Dense(len(categories), activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# =================== EARLY STOP ===================
early_stop = EarlyStopping(patience=3, restore_best_weights=True)

# =================== TRAIN AND EVALUATE ALL MODELS ===================
# CNN
print("\n🧠 Training CNN...")
cnn_model = build_cnn_model()
cnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, callbacks=[early_stop])
print("📊 CNN Results:")
evaluate_model(cnn_model, X_val, y_val)

# RNN
print("\n🧠 Training RNN...")
rnn_model = build_rnn_model()
rnn_model.fit(X_train_rnn, y_train, validation_data=(X_val_rnn, y_val), epochs=10, callbacks=[early_stop])
print("📊 RNN Results:")
evaluate_model(rnn_model, X_val_rnn, y_val)

# Bidirectional RNN
print("\n🧠 Training Bidirectional RNN...")
bidir_rnn_model = build_bidirectional_rnn_model()
bidir_rnn_model.fit(X_train_rnn, y_train, validation_data=(X_val_rnn, y_val), epochs=10, callbacks=[early_stop])
print("📊 Bidirectional RNN Results:")
evaluate_model(bidir_rnn_model, X_val_rnn, y_val)

# LSTM
print("\n🧠 Training LSTM...")
lstm_model = build_lstm_model()
lstm_model.fit(X_train_rnn, y_train, validation_data=(X_val_rnn, y_val), epochs=10, callbacks=[early_stop])
print("📊 LSTM Results:")
evaluate_model(lstm_model, X_val_rnn, y_val)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Loaded 924 images with shape: (924, 128, 128, 3)

🧠 Training CNN...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.1390 - loss: 6.0191 - val_accuracy: 0.2541 - val_loss: 2.4177
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 38s 986ms/step - accuracy: 0.2790 - loss: 2.3823 - val_accuracy: 0.2541 - val_loss: 2.2961
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.2587 - loss: 2.2706 - val_accuracy: 0.2541 - val_loss: 2.2001
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 22s 912ms/step - accuracy: 0.2953 - loss: 2.1795 - val_accuracy: 0.3081 - val_loss: 2.1416
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 24s 991ms/step - accuracy: 0.3161 - loss: 2.0644 - val_accuracy: 0.3568 - val_loss: 2.0197
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 23s 973ms/step - accuracy: 0.3437 - loss: 2.0034 - val_accuracy: 0.3946 - val_loss: 1.9618
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.3779 - loss: 1.8962 - val_accuracy: 0.4595 - val_loss: 1.8240
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 38s 924ms/step - accuracy: 0.4131 - loss: 1.7983 - val_accuracy: 0.3

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step
Accuracy : 0.2864864864864865
Precision: 0.24348660348660348
Recall   : 0.2864864864864865
F1 Score : 0.20842995768269376

🧠 Training LSTM...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 355ms/step - accuracy: 0.2158 - loss: 2.0947 - val_accuracy: 0.2541 - val_loss: 1.9870
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 243ms/step - accuracy: 0.2552 - loss: 1.9742 - val_accuracy: 0.2486 - val_loss: 1.9338
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 329ms/step - accuracy: 0.2886 - loss: 1.9231 - val_accuracy: 0.2757 - val_loss: 1.9435
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 271ms/step - accuracy: 0.3115 - loss: 1.8903 - val_accuracy: 0.2811 - val_loss: 1.9229
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 235ms/step - accuracy: 0.3110 - loss: 1.8718 - val_accuracy: 0.2811 - val_loss: 1.8707
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 237ms/step - accuracy: 0.3672 - loss: 1.8230 - val_accuracy: 0.2703 - val_loss: 1.8524
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 298ms/step - accuracy: 0.3545 - loss: 1.7744 - val_accuracy: 0.2919 - val_loss: 1.8537
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 317ms/step - accuracy: 0.3691 - loss: 1.7432 - val_accurac

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 227ms/step
Accuracy : 0.3081081081081081
Precision: 0.3125977094829554
Recall   : 0.3081081081081081
F1 Score : 0.2553207011364585


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
